### Discovery Significance vs Mass

This notebook makes a p0 plot a scan of the discovery significance vs mass for the H->gg case


We read first the file with the workspace and we retrieve the needed objects

In [1]:
  // Check if example input file exists
  TFile *file = TFile::Open("HiggsModel.root");

  // get the workspace out of the file
  RooWorkspace* w = (RooWorkspace*) file->Get("w");


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [8]:
 // get the data out of the file
  RooAbsData* data2 = w->data("data");

In [ ]:
  // get the modelConfig (S+B) out of the file
  // and create the B model from the S+B model
  RooStats::ModelConfig*  sbModel = (RooStats::ModelConfig*) w->obj("ModelConfig");
  sbModel->SetName("S+B Model");      
  RooRealVar* poi = (RooRealVar*) sbModel->GetParametersOfInterest()->first();
  poi->setVal(200);  // set POI snapshot in S+B model for expected significance
  sbModel->SetSnapshot(*poi);
  RooStats::ModelConfig * bModel = (RooStats::ModelConfig*) sbModel->Clone();
  bModel->SetName("B Model");      
  poi->setVal(0);
  bModel->SetSnapshot( *poi  );

In [1]:
vector<double> masses;
vector<double> p0values;
vector<double> p0valuesExpected;
double massMin, massMax; 

###   Loop on the mass values

Perform a loop between the given values and perform an asymptotic hypothesis test to compute the p-value at each scanned mass point

In [ ]:
massMin = 112;
massMax = 158;


int npoints = 30;
for( double mass=massMin; mass<=massMax; mass += (massMax-massMin)/double(npoints) )
  {
      
      
     cout << endl << endl << "Running for mass: " << mass << endl << endl;
     w->var("mass")->setVal( mass );

     RooStats::AsymptoticCalculator ac(*data2, *sbModel, *bModel);
     ac.SetOneSidedDiscovery(true);  // for one-side discovery test                                      
     RooStats::AsymptoticCalculator::SetPrintLevel(-1);


     RooStats::HypoTestResult* asymCalcResult = ac.GetHypoTest();
 
     asymCalcResult->Print();
     
     masses.push_back( mass );
     p0values.push_back( asymCalcResult->NullPValue() );
  
     double expectedP0 = RooStats::AsymptoticCalculator::GetExpectedPValues(  asymCalcResult->NullPValue(),  asymCalcResult->AlternatePValue(), 0, false);
     p0valuesExpected.push_back( expectedP0 );
     std::cout << "expected p0 = " << expectedP0 << std::endl;

  }



RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Running for mass: 0



IncrementalExecutor::executeFunction: symbol '_ZNSt3__1L4moveIRdEEONS_16remove_referenceIT_E4typeEOS3_' unresolved while linking function '_GLOBAL__sub_I_cling_module_604'!
You are probably missing the definition of std::__1::remove_reference<double&>::type&& std::__1::move<double&>(double&)
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime8internal9EvaluateTIvEET_PNS1_15DynamicExprInfoEPN5clang11DeclContextE' unresolved while linking function '_GLOBAL__sub_I_cling_module_604'!
You are probably missing the definition of void cling::runtime::internal::EvaluateT<void>(cling::runtime::internal::DynamicExprInfo*, clang::DeclContext*)
Maybe you need to load the corresponding shared library?
input_line_33:2:39: error: use of undeclared identifier 'data2'
 new RooStats::AsymptoticCalculator(* data2, * sbModel, * bModel)
                                      ^
input_line_33:2:48: error: use of undeclared identifier 'sbModel

Create a plot with the obtained p-values

In [6]:
  TGraph * graph1  = new TGraph(masses.size(),&masses[0],&p0values[0]);
  TGraph * graph2  = new TGraph(masses.size(),&masses[0],&p0valuesExpected[0]);

  graph1->SetMarkerStyle(20);
  graph1->Draw("APC");
  graph2->SetLineStyle(2);
  graph2->Draw("C");
  graph1->GetXaxis()->SetTitle("mass");
  graph1->GetYaxis()->SetTitle("p0 value");
  graph1->SetTitle("Significance vs Mass");
  graph1->SetMinimum(graph2->GetMinimum());
  graph1->SetLineColor(kBlue);
  graph2->SetLineColor(kRed);
  gPad->SetLogy(true);

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [ ]:
gPad->Draw()